In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import string
import re
import spacy
from collections import Counter
import pickle

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#### Loading Images and Captions

In [ ]:
with open('/content/drive/My Drive/Courses/DeepLearning/HW03/Q03/Datas/Flickr8k.token.txt') as f:
    lines = f.readlines()
caption_dict = {}
image_dict = {}
for line in lines:
  try:
    image_path = line.split('#')[0]
    caption = line.split('#')[1]
    image = plt.imread("/content/drive/My Drive/Courses/DeepLearning/HW03/Q03/Datas/Flicker8k_Dataset/"+image_path)
    image_dict[image_path] = image
    caption_dict[image_path] =  caption
  except:
    pass

In [ ]:
with open('/content/drive/My Drive/Courses/DeepLearning/HW03/Q03/Datas/caption_dict.pkl', 'wb') as f:
    pickle.dump(caption_dict, f)  # save it

with open('/content/drive/My Drive/Courses/DeepLearning/HW03/Q03/Datas/image_dict.pkl', 'wb') as f:
    pickle.dump(image_dict, f)  # save it

In [ ]:
# Train Datas
with open('/content/drive/My Drive/Courses/DeepLearning/HW03/Q03/Datas/Flickr_8k.trainImages.txt') as f:
  lines = f.readlines()
datas = pd.DataFrame(columns=["caption","image"])

for line in lines:
  line = line.replace('\n', '')
  image = image_dict[line]
  caption = caption_dict[line]
  datas = datas.append({"caption":caption, "image":image}, ignore_index=True)

# Test Datas
with open('/content/drive/My Drive/Courses/DeepLearning/HW03/Q03/Datas/Flickr_8k.testImages.txt') as f:
  lines = f.readlines()

for line in lines:
  line = line.replace('\n', '')
  image = image_dict[line]
  caption = caption_dict[line]
  datas = datas.append({"caption":caption, "image":image}, ignore_index=True)

del caption_dict
del image_dict
datas.head(4)

#### Preprocessing the datas

In [ ]:
with open('/content/drive/My Drive/Courses/DeepLearning/HW03/Q03/Datas/datas.pkl', 'wb') as f:
    pickle.dump(datas, f)  # save it

In [ ]:
with open('/content/drive/My Drive/Courses/DeepLearning/HW03/Q03/Datas/datas.pkl', 'rb') as f:
    datas = pickle.load(f) # load it

In [ ]:
tok = spacy.load('en')
def tokenize (text):
    text = re.sub(r"[^\x00-\x7F]+", " ", text)
    regex = re.compile('[' + re.escape(string.punctuation) + '0-9\\r\\t\\n]')
    nopunct = regex.sub(" ", text.lower())
    return [token.text for token in tok.tokenizer(nopunct)]

In [ ]:
# remove punctuation and numbers and unfrequent words
datas_preprocessed = datas.copy()
counts = Counter()
for index, row in datas_preprocessed.iterrows():
    counts.update(tokenize(row['caption']))

print("num_words before:",len(counts.keys()))
for word in list(counts):
    if counts[word] < 10:
      datas_preprocessed['caption'] = datas_preprocessed['caption'].str.replace("\b{0}\b".format(word), '')
      del counts[word]

punctuation = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '\t',"\n",'#','@','.',',','?','!',')', '(']
for char in punctuation:
    datas_preprocessed['caption'] = datas_preprocessed['caption'].str.replace(char, '')

print("num_words after:",len(counts.keys()))

datas_preprocessed.head(4)

num_words before: 4079
num_words after: 664


,caption,image
0,Two dogs running through a low lying body of w...,"[[[38, 31, 25], [64, 50, 49], [78, 73, 67], [2..."
1,The little boy is playing with a croquet hamme...,"[[[254, 254, 254], [254, 254, 254], [254, 254,..."
2,A dog with something pink in its mouth is look...,"[[[146, 143, 154], [149, 143, 155], [153, 142,..."
3,The large brown dog is running on the beach by...,"[[[121, 158, 202], [117, 154, 198], [118, 155,..."
